In [ ]:
# 6-3 Moneybox 클래스를 작성하라. 이 클래스는 저축한 총액을 저장하는 인스턴스 변수가 있다.
# deposit(money) 메서드는 인수로 주어진 금액만큼 저축하고 총액을 출력한다,
# extract(money) 메소드는 money를 인출하고 총액을 출력한다. get_total() 메소드는 총액을 알려준다.

# my_money = Moneybox() 객체를 생성하고 1000원을 저축하고, 200원을 인출한 다음, 총액을 출력하라.

class Moneybox: 
    def __init__(self):
        self.total = 0  # 저축한 총액을 저장하는 인스턴스 변수

    def deposit(self, money):
        self.total += money  # money만큼 저축
        print(f"Deposited: {money}, Total: {self.total}")

    def extract(self, money):
        if money > self.total:
            print("Insufficient funds!")
        else:
            self.total -= money  # money만큼 인출
            print(f"Extracted: {money}, Total: {self.total}")

    def get_total(self):
        return self.total  # 총액 반환

# 객체 생성
my_money = Moneybox()
# 1000원 저축
my_money.deposit(1000)
# 200원 인출
my_money.extract(200)
# 총액 출력
print(f"Total amount: {my_money.get_total()}")

In [ ]:
# 6-4 Password_manager 클래스를 작성하라. 이 클래스는 사용자의 모든 과거 패스워드를 저장하
# 는 old_passwords 리스트 변수를 갖는다. 리스트의 마지막 요소는 현재 패스워드이다. 현재 패
# 스워드를 알려주는 get_password와 새로운 패스워드를 설정하는 set_password 메소드를 정의하라. set_password는 과거에 사용하지 않은 새로운 패스워드를 설정할 때만 유효하다. 문자열을 입력받아 현재 패스워드와 같은지 True 또는 False를 반환하는 is_correct 메소드를 정의하라

class Password_manager:
    def __init__(self):
        self.old_passwords = []  # 과거 패스워드를 저장하는 리스트

    def get_password(self):
        if self.old_passwords:
            return self.old_passwords[-1]  # 현재 패스워드 반환
        return None

    def set_password(self, new_password):
        if new_password in self.old_passwords:
            print("This password has been used before. Choose a different one.")
        else:
            self.old_passwords.append(new_password)  # 새로운 패스워드 설정
            print("Password updated successfully.")

    def is_correct(self, password):
        return password == self.get_password()  # 현재 패스워드와 비교하여 True/False 반환
    
# 객체 생성
pm = Password_manager()
# 패스워드 설정
pm.set_password("mypassword123")
# 현재 패스워드 확인
print(f"Current password: {pm.get_password()}")

In [ ]:
#6-5 Articles 클래스를 작성하라. 이 클래스는 각각 제품의 이름, 재고 수량, 단가를 나타내는
# name, stock, price 인스턴스 변수를 가지고 있다. offer_price() 메소드는 구매 수량을 인수
# 로 전달받아 총 가격을 반환한다. 구매 수량이 1~10개 이하이면 정상 가격을 반환하고, 11~50개
# 이면 5% 할인된 가격을 반환한다. 또 51개 이상이면 10% 할인해 준다. purchase() 메소드는
# 구매 수량을 인수로 전달받아 재고 수량을 구매 수량만큼 감소한다.


# 이 클래스를 이용하여 단가 1000원인 ”mask“를 10,000장으로 설정하고, 40장 구매 가격과 구매 후 재고를 출력하라.

class Articles:
    def __init__(self, name, stock, price):
        self.name = name
        self.stock = stock
        self.price = price

    def offer_price(self, quantity):
        if quantity <= 10:
            total_price = self.price * quantity
        elif 11 <= quantity <= 50:
            total_price = self.price * quantity * 0.95
        else:
            total_price = self.price * quantity * 0.90
        return total_price

    def purchase(self, quantity):
        if quantity <= self.stock:
            self.stock -= quantity
        else:
            print("재고가 부족합니다.")

# 단가 1000원인 "mask"를 10,000장으로 설정
mask = Articles("mask", 10000, 1000)
# 40장 구매 가격과 구매 후 재고 출력
purchase_quantity = 40
total_price = mask.offer_price(purchase_quantity)
mask.purchase(purchase_quantity)
print(f"구매 가격: {total_price}원")
print(f"구매 후 재고: {mask.stock}장")

In [ ]:
# 6-6 다음 프로그램을 실행했을 때 아래 내용이 출력되도록 클래스를 정의하라. 학번(id) 앞 4자리가 2018보다 작으면 grade() 메소드에 의해 “3학년입니다“가 출력되고 아니면 ”3학년이 아닙니다“가 출력된다.

class Student:
    def __init__(self, id):
        self.id = id

    def grade(self):
        if int(self.id[:4]) < 2018:
            print("3학년입니다")
        else:
            print("3학년이 아닙니다")
student1 = Student("20170001")
student1.grade()  # 출력: 3학년입니다
student2 = Student("20180001")
student2.grade()  # 출력: 3학년이 아닙니다

In [ ]:
# 6-7 다음 Person 클래스를 상속 받는 Employee 클래스를 정의하라. Employee 클래스에 employeeID 속성을 추가하고 getID() 메소드를 정의하라. getID() 메소드는 employeeID를 반환하는 메소드이다. 
# Employee 클래스를 이용하여 Employee("동양", 65, 2019)로 생성된 객체의 이름, 나이, ID를 출력하라.

class person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    def getName(self):
        return self.name
    def getAge(self):
        return self.age
class Employee(person):
    def __init__(self, name, age, employeeID):
        super().__init__(name, age)
        self.employeeID = employeeID
    def getId(self):
        return self.employeeID
emp = Employee("동양", 65, 2019)
print(emp.getName())
print(emp.getAge())
print(emp.getId())

In [ ]:
# 6-8 앞의 Person 클래스에서 새로운 Person 객체가 생성될 때 ID를 1001번지부터 순차적으로 자동으로 부여할 수 있도록 Employee 클래스를 포함해서 변경하시오. 새로운 Employee객체를
# 만들어 getId() 를 호출할 때 Id가 중복되면 안된다.

class person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        person.id += 1
        self.employeeID = person.id
    def getName(self):
        return self.name
    def getAge(self):
        return self.age
class Employee(person):
    def __init__(self, name, age, employeeID):
        super().__init__(name, age)
        self.employeeID = employeeID
    def getId(self):
        return self.employeeID
    
person.id = 1000
e1 = Employee("kim", 25, person.id)
print(e1.getName(), e1.getAge(), e1.getId())
e2 = Employee("lee", 30, person.id)
print(e2.getName(), e2.getAge(), e2.getId())
e3 = Employee("park", 35, person.id)
print(e3.getName(), e3.getAge(), e3.getId())
e4 = Employee("choi", 40, person.id)
print(e4.getName(), e4.getAge(), e4.getId())
e5 = Employee("jung", 45, person.id)
print(e5.getName(), e5.getAge(), e5.getId())

In [ ]:
# 6-9 파이썬 언어에서 다음에 대해 조사하고 예제코드를 실행하여 보라
# 1. 파이썬으로 싱글톤 패턴을 구현하는 방법들

# Mixin 클래스를 이용한 싱글톤 패턴 구현 예제
class SingletonMixin:
    _instances = {}

    @classmethod
    def instance(cls, *args, **kwargs):
        if cls not in cls._instances:
            print(f"{cls.__name__} 객체 새로 생성")
            cls._instances[cls] = cls(*args, **kwargs)
        else:
            print(f"{cls.__name__} 기존 객체 반환")
        return cls._instances[cls]


class BaseClass:
    pass

class MyClass(BaseClass, SingletonMixin):
    pass

class OtherClass(BaseClass, SingletonMixin):
    pass


# 실행 예시
a1 = MyClass.instance()
a2 = MyClass.instance()
b1 = OtherClass.instance()
b2 = OtherClass.instance()

print(a1 is a2)  
print(b1 is b2)  
print(a1 is b1)

# __new__ 메서드를 이용한 싱글톤 패턴 구현 예제
class Singleton(object):
    _instance = None
    def __new__(class_, *args, **kwargs):
        if not isinstance(class_._instance, class_):
            class_._instance = super().__new__(class_)
        return class_._instance


class BaseClass:
    pass

class MyClass(Singleton, BaseClass):
    pass

class OtherClass(Singleton):
    pass

a1 = MyClass()
a2 = MyClass()
b1 = OtherClass()
b2 = OtherClass()

print(a1 is a2)  
print(b1 is b2)  
print(a1 is b1)  


# 2. 파이썬에서 멀티쓰레드 구현하는 기법
from threading import Thread

def work(id, start, end, result):
    total = 0
    for i in range(start, end):
        total += i
    result.append(total)
    return

if __name__ == "__main__":
    START, END = 0, 100000000
    result = list()
    th1 = Thread(target=work, args=(1, START, END, result))
    
    th1.start()
    th1.join()

print(f"Result: {sum(result)}")
# 3. 멀티쓰레드에서 공유 변수에 대한 Critical Section 을 보호하는 기법
import threading
import time

rlock = threading.RLock()
cond = threading.Condition(rlock)
queue = []

def consumer(name):
    with cond:
        while not queue:
            print(f"[{name}] 큐 비어 있음 → wait() 진입")
            cond.wait()  # 여기서 잠듦, producer가 notify할 때까지
        item = queue.pop(0)
        print(f"[{name}] 아이템 소비: {item}")

def producer(item):
    with cond:
        print(f"[Producer] 아이템 생산: {item}")
        queue.append(item)
        cond.notify()  # 대기 중인 소비자 하나 깨움


# 실행
t1 = threading.Thread(target=consumer, args=("C1",))
t2 = threading.Thread(target=consumer, args=("C2",))

t1.start()
t2.start()

time.sleep(1)  # 소비자들이 먼저 기다리도록 잠깐 대기

producer("apple")
time.sleep(1)
producer("banana")

t1.join()
t2.join()




MyClass 객체 새로 생성
MyClass 기존 객체 반환
OtherClass 객체 새로 생성
OtherClass 기존 객체 반환
True
True
False
True
True
False
Result: 4999999950000000
[C1] 큐 비어 있음 → wait() 진입
[C2] 큐 비어 있음 → wait() 진입
[Producer] 아이템 생산: apple
[C1] 아이템 소비: apple
[Producer] 아이템 생산: banana
[C2] 아이템 소비: banana


In [24]:
# 6-10 동물(Animal)을 울음을 추상 메소드로 하는 추상클래스로 정의하고,
# 개(Dog)와 고양이(Cat)를 구체적인 클래스로 정의한 후,
# 각 동물의 울음소리를 출력하는 프로그램을 작성하시오.
# 각 동물객체는 자신의 이름을 속성으로 가지며,
# 생성자에서 이름을 초기화합니다.
# 추가적으로 Animal 클래스는 현재 생성된 동물 객체의 수를 클래스 변수로 가지고,
# 정적 메소드로 현재 동물 객체의 수를 반환하는 메소드를 구현
# 객체가 생성(__init__)될 때 객체의 수를 증가하고 삭제(__del__(self) ) 될 때 객체의 수를 감소

from abc import ABC, abstractmethod

class Animal(ABC):
    _count = 0  # 클래스 변수: 현재 생성된 동물 객체 수

    @staticmethod
    def get_count():
        return Animal._count  # 정적 메서드: 현재 동물 수 반환

    def __init__(self, name: str):
        self.name = name               # 각 동물 객체는 이름 속성 보유
        Animal._count += 1             # 객체 생성 시 개수 증가

    def __del__(self):
        # 주의: __del__ 호출 시점은 인터프리터/GC에 따라 달라질 수 있음
        Animal._count -= 1             # 객체 소멸 시 개수 감소

    @abstractmethod
    def sound(self) -> str:
        """각 동물의 울음소리를 문자열로 반환"""
        pass


class Dog(Animal):
    def sound(self) -> str:
        return "멍멍"


class Cat(Animal):
    def sound(self) -> str:
        return "야옹"


# === 사용 예시 ===
d = Dog("뽀삐")
c = Cat("나비")

# 각 동물의 울음소리 출력 (이름 포함)
print(f"{d.name}: {d.sound()}")  # 뽀삐: 멍멍
print(f"{c.name}: {c.sound()}")  # 나비: 야옹

# 현재 동물 객체 수
print("현재 동물 수:", Animal.get_count())




뽀삐: 멍멍
나비: 야옹
현재 동물 수: 0
